In [31]:
import os
import hashlib
import tensorflow as tf
from PIL import Image
from object_detection.utils import dataset_util


In [44]:
#annotations_file = r'C:\Users\1\Desktop\сase1\set1test\test_annotations.txt'
annotations_file = r'C:\Users\1\Desktop\сase1\set1train\train_annotations.txt'

images_directory = r'C:\Users\1\Desktop\сase1\set1train'
#images_directory = r'C:\Users\1\Desktop\сase1\set1test'

output_path = r'C:\Users\1\Desktop\сase1\outputtrain.tfrecord'
#output_path = r'C:\Users\1\Desktop\сase1\outputtest.tfrecord'

class_labels = {} 

In [45]:
def create_tf_example(image_path, signs_info):
    with tf.io.gfile.GFile(image_path, 'rb') as fid:
        encoded_image = fid.read()
    
    image = Image.open(image_path)
    width, height = image.size

    tf_example = tf.train.Example(features=tf.train.Features(feature={
        'image/height': dataset_util.int64_feature(height),
        'image/width': dataset_util.int64_feature(width),
        'image/filename': dataset_util.bytes_feature(os.path.basename(image_path).encode('utf8')),
        'image/source_id': dataset_util.bytes_feature(os.path.basename(image_path).encode('utf8')),
        'image/encoded': dataset_util.bytes_feature(encoded_image),
        'image/format': dataset_util.bytes_feature(b'jpeg'),
    }))
    
    if not signs_info or len(signs_info[0]) == 0:
        return tf_example

    if "MISC_SIGNS" in signs_info[0]:
        signs_info = signs_info[0].split(';')[1:] 
        

    for sign_data in signs_info:
        sign_parts = sign_data.split(',')
        if len(sign_parts) >= 6:
            class_name = sign_parts[6].strip()
            if class_name not in class_labels:
                class_labels[class_name] = len(class_labels) + 1
                
    def save_class_labels(class_labels, output_file):
        with open(output_file, 'w', encoding='utf-8') as f:
            for class_name, class_id in class_labels.items():
                f.write("item {\n")
                f.write(f"\tid: {class_id}\n")
                f.write(f"\tname: '{class_name}'\n")
                f.write("}\n")
    save_class_labels(class_labels, 'labels.pbtxt')            

    for sign_data in signs_info:
        sign_parts = sign_data.split(',')
        if len(sign_parts) < 6: 
            continue
        
        visibility = sign_parts[0]  
        coordinates = list(map(float, sign_parts[1:5]))
        class_name = sign_parts[6]

        normalized_coordinates = [coord / width if idx % 2 == 0 else coord / height for idx, coord in enumerate(coordinates)]

        tf_example.features.feature['image/object/bbox/xmin'].float_list.value.extend([normalized_coordinates[0]])
        tf_example.features.feature['image/object/bbox/ymin'].float_list.value.extend([normalized_coordinates[1]])
        tf_example.features.feature['image/object/bbox/xmax'].float_list.value.extend([normalized_coordinates[2]])
        tf_example.features.feature['image/object/bbox/ymax'].float_list.value.extend([normalized_coordinates[3]])
        tf_example.features.feature['image/object/class/text'].bytes_list.value.extend([class_name.encode('utf8')])

        if class_name in class_labels:
            class_label = class_labels[class_name]
            tf_example.features.feature['image/object/class/label'].int64_list.value.extend([class_label])
    
    return tf_example


def main():
    writer = tf.io.TFRecordWriter(output_path)

    with open(annotations_file, 'r') as file:
        lines = file.readlines()

    for line in lines:
        parts = line.strip().split(':')
        image_name = parts[0]
        signs_info = parts[1] if len(parts) > 1 else ''

        if not signs_info:
            continue

        image_path = os.path.join(images_directory, image_name)
        signs = signs_info.split(';')

        tf_example = create_tf_example(image_path, signs)
        writer.write(tf_example.SerializeToString())

    writer.close()

if __name__ == "__main__":
    main()


In [ ]:
cd ~/models/research
python3 object_detection/legacy/train.py --logtostderr --pipeline_config_path object_detection/rfcn_resnet101/rfcn_resnet101_gtsdb3.config --train_dir object_detection/rfcn_resnet101/inference_graph
